In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
from brian2 import *
from brian2tools import *
import numpy as np
import pandas as pd


# Customize matplotlib
matplotlib.rcParams.update(
    {
        'text.usetex': False,
        'font.family': 'stixgeneral',
        'mathtext.fontset': 'stix',
        'font.size': 22,
        'ytick.color' : 'black',
        "xtick.color" : 'black',
        'axes.labelcolor' : 'black',
        'axes.edgecolor' : 'black'
	}
)

INFO:numexpr.utils:NumExpr defaulting to 8 threads.


In [2]:
n_points = 51

vec_alpha = np.linspace(0, 1, n_points)

EL = -60*mV
EE = 0*mV
EI = -80*mV
tauA = 1*ms
tauN = 100*ms
tauI = 10*ms
tauL = 20*ms
vt = -50*mV



eqs = '''
dv/dt = ((EL - v) + ((1 - alpha)*gA + alpha*gN)*(EE - v) + gI*(EI - v))/tauL : volt (unless refractory)
dgA/dt = -gA/tauA : 1
dgN/dt = -gN/tauN : 1
dgI/dt = -gI/tauI : 1
alpha : 1
'''

rate_list = [5, 10, 20]
firing_rate_alpha = {'alpha':vec_alpha, 5:np.zeros(n_points), 10:np.zeros(n_points), 20:np.zeros(n_points)}
CVISI_alpha = {'alpha':vec_alpha, 5:np.zeros(n_points), 10:np.zeros(n_points), 20:np.zeros(n_points)}

for rate in rate_list:
	print(rate, end='\t')
	for i, alpha_value in enumerate(vec_alpha):
		start_scope()
		print(i/size(vec_alpha), end=' ')
		defaultclock.dt = 0.01*ms
		G = NeuronGroup(1, eqs, threshold='v>vt', reset='v=EL', refractory=2*ms, method='rk4')
		GPA = PoissonGroup(400, rates=rate*Hz)
		GPN = PoissonGroup(400, rates=rate*Hz)
		GPI = PoissonGroup(100, rates=rate*Hz)

		SA = Synapses(GPA, G, 'w : 1', on_pre='gA += w')
		SN = Synapses(GPN, G, 'w : 1', on_pre='gN += w')
		SI = Synapses(GPI, G, 'w : 1', on_pre='gI += w')
		SA.connect()
		SN.connect()
		SI.connect()

		G.v = EL
		SA.w = 0.1
		SN.w = 0.1
		SI.w = 0.4

		G.alpha = alpha_value
			
		run(5000*ms)
		
		spkM = SpikeMonitor(G)

		run(10000*ms)
		
		train = spkM.spike_trains()
		ISI = np.diff(train[0])

		firing_rate_alpha[rate][i] = spkM.num_spikes/10.0
		CVISI_alpha[rate][i] = ISI.var()/ISI.mean()**2
    
	print('\n')

5	0.0 

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/brian2/units/fundamentalunits.py:252: RuntimeWarning: Degrees of freedom <= 0 for slice
  return Quantity(func(ar, *args, **kwds),
 [py.warnings]
  return Quantity(func(ar, *args, **kwds),

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/numpy/core/_methods.py:222: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',
 [py.warnings]
  arrmean = um.true_divide(arrmean, div, out=arrmean, casting='unsafe',

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/numpy/core/_methods.py:256: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
 [py.warnings]
  ret = ret.dtype.type(ret / rcount)

WARNING    /home/marcelo/miniconda3/envs/worklab/lib/python3.10/site-packages/brian2/units/fundamentalunits.py:229: RuntimeWarning: Mean of empty slice.


0.0196078431372549 0.0392156862745098 0.058823529411764705 0.0784313725490196 0.09803921568627451 0.11764705882352941 0.13725490196078433 0.1568627450980392 0.17647058823529413 0.19607843137254902 0.21568627450980393 0.23529411764705882 0.2549019607843137 0.27450980392156865 0.29411764705882354 0.3137254901960784 0.3333333333333333 0.35294117647058826 0.37254901960784315 0.39215686274509803 0.4117647058823529 0.43137254901960786 0.45098039215686275 0.47058823529411764 0.49019607843137253 0.5098039215686274 0.5294117647058824 0.5490196078431373 0.5686274509803921 0.5882352941176471 0.6078431372549019 0.6274509803921569 0.6470588235294118 0.6666666666666666 0.6862745098039216 0.7058823529411765 0.7254901960784313 0.7450980392156863 0.7647058823529411 0.7843137254901961 0.803921568627451 0.8235294117647058 0.8431372549019608 0.8627450980392157 0.8823529411764706 0.9019607843137255 0.9215686274509803 0.9411764705882353 0.9607843137254902 0.9803921568627451 

10	0.0 0.0196078431372549 0.039

In [3]:
fRate_data_path = "../../Data/fRate_data/"
V_data_path = "../../Data/V_data/"

# Convert to pandas DataFrame
df_firing_rate = pd.DataFrame.from_dict(firing_rate_alpha)
df_CVISI = pd.DataFrame.from_dict(CVISI_alpha)

# Save Dataframe do csv file
df_firing_rate.to_csv(f'{fRate_data_path}firing_rate_iRate_interpolated.dat')
df_CVISI.to_csv(f'{fRate_data_path}CVISI_iRate_interpolated.dat')


In [4]:
df_firing_rate

,alpha,5,10,20
0,0.00,0.0,0.0,0.0
1,0.02,0.0,0.0,0.0
2,0.04,10.7,17.3,15.5
3,0.06,55.6,124.2,193.8
4,0.08,129.4,217.4,313.1
5,0.10,184.6,278.1,362.2
6,0.12,222.0,316.0,389.3
7,0.14,255.1,340.9,407.3
8,0.16,279.1,360.5,421.0
9,0.18,298.1,375.3,430.4
